## Smooth Trend Model


## Install Packages

In [ ]:
!pip install numpyro

【重要】パッケージのインストール完了後に、ランタイムを再起動して下さい！

## Import Packages

In [ ]:
import numpyro
import numpyro.distributions as dist

import jax
import jax.numpy as jnp

import arviz as az

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from numpyro.contrib.control_flow import scan

In [ ]:
plt.rcParams['font.size'] = 14
plt.rcParams['figure.figsize'] = (8, 5)

In [ ]:
numpyro.set_platform('cpu')
numpyro.set_host_device_count(4)

## Load Data

In [ ]:
data = pd.read_csv('data.csv')

t = data['t']
u = data['u']
y = data['y']

plt.plot(u)
plt.plot(y, '.');

## Define Model & Inference

In [ ]:
y = jnp.array(y)

In [ ]:
def gaussian_random_walk(name, num_steps, scale=1.0):

    def transition_fn(u_prev, t):
        
        u_curr = numpyro.sample(name, dist.Normal(u_prev, scale))
        
        return u_curr, u_curr
    
    t = jnp.arange(num_steps)
    
    _, u = scan(transition_fn, 0.0, t)
    
    return u

In [ ]:
def model(y_obs, future=0):
    
    num_steps = len(y_obs)
    
    #
    # System Model
    #
    
    sd_b = numpyro.sample('sd_b', dist.HalfNormal(10))

    c = numpyro.sample('c', dist.Normal(10))
    b = gaussian_random_walk('b', num_steps + future, scale=sd_b)
    u = numpyro.deterministic('u', c + jnp.cumsum(b))
    
    #
    # Observer
    #
    
    sd_y = numpyro.sample('sd_y', dist.HalfNormal(10))

    
    with numpyro.handlers.condition(data={'y':y_obs}):
                        
        def observer_fn(carry, uu):
            
            y = numpyro.sample('y', dist.Normal(uu, sd_y))
            
            return carry, y
                
        _, y = scan(observer_fn, None, u)
        

    if future > 0:
        
        numpyro.deterministic('y_pred', y[-future:])

In [ ]:
nuts = numpyro.infer.NUTS(model, target_accept_prob=0.95)
mcmc = numpyro.infer.MCMC(nuts, num_warmup=1000, num_samples=1000, num_chains=4)

mcmc.run(jax.random.PRNGKey(1), y)

trace = mcmc.get_samples()

In [ ]:
idata = az.from_numpyro(mcmc)

In [ ]:
az.summary(idata, var_names=['c', 'sd_b', 'sd_y'])

In [ ]:
az.plot_trace(idata, var_names=['c', 'sd_b', 'sd_y'])
plt.gcf().subplots_adjust(wspace=0.5, hspace=0.5)

## Check Estimated Trend

In [ ]:
u_sampled = trace['u']

In [ ]:
mu = jnp.mean(u_sampled, 0)
pi = jnp.percentile(u_sampled, jnp.array([5, 95]), 0)

In [ ]:
plt.plot(t, y, '.', color='C1')

plt.plot(t, mu, '-.')
plt.fill_between(t, pi[0, :], pi[1, :], alpha=0.3);

## Check Prediction

In [ ]:
observed_steps = len(y)
future_steps = 50

In [ ]:
predictive = numpyro.infer.Predictive(model, trace, return_sites=['y_pred'])
ppc_samples = predictive(jax.random.PRNGKey(2), y, future=future_steps)

In [ ]:
y_pred = ppc_samples['y_pred']
t_pred = np.arange(observed_steps, observed_steps + future_steps)

In [ ]:
mu_pred = jnp.mean(y_pred, 0)
pi_pred = jnp.percentile(y_pred, jnp.array([5, 95]), 0)

In [ ]:
plt.plot(t, y)

plt.plot(t_pred, mu_pred, '-.')
plt.fill_between(t_pred, pi_pred[0, :], pi_pred[1, :], alpha=0.3);

In [ ]:
plt.plot(t, y)
plt.plot(t_pred, y_pred[:5,].T);